In [2]:
import sqlite3
import requests
import json
import google.generativeai as genai
from base64 import b64encode
import os
from dotenv import load_dotenv
import numpy as np

load_dotenv()

True

In [3]:
def edamam_get_recipe(dish_name):
    url = "https://api.edamam.com/api/recipes/v2"
    query = {
        "type": "public",
        "q": dish_name,
        "app_id": os.getenv("EDAMAM_APP_ID"),
        "app_key": os.getenv("EDAMAM_API_KEY"),
        'ingr': 20,
        'beta': False,

    }
    response = requests.get(url, params=query)
    if response.status_code == 200:
        return response.json()
    else:
        print(response.status_code)
        return None


def get_description(dish_name, dish_ingredients):
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(f'Describe the the next dish in no more than two lines:'
                                      f'{dish_name} with {", ".join([ingredient.get("name") for ingredient in dish_ingredients])}')

    return response.text


def clean_unit(unit):
    if unit is None:
        return ''
    return unit.replace('<', '').replace('>', '').replace('(', '').replace(')', '').capitalize()


def get_embeddings(description):
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    result = genai.embed_content(
        model="models/embedding-001",
        content=description,
        task_type="semantic_similarity")

    return result.get('embedding')


def edamam_to_dish(recipe):
    name = recipe.get('label')
    # image = recipe.get('image')

    image_url = recipe.get('image')
    response = requests.get(image_url)  # Download the image
    image = b64encode(response.content).decode('utf-8')  # Encode the image to base64

    ingredients = []
    ingredients_names = []
    for ingredient in recipe.get('ingredients'):
        if ingredient.get('food') in ingredients_names:
            print(f'Found duplicate ingredient: {ingredient.get("food")}')
            ingredients[ingredients_names.index(ingredient.get('food'))]['amount'] += ingredient.get('quantity')
        else:
            ingredients_names.append(ingredient.get('food'))
            ingredients.append({
                'name': ingredient.get('food').capitalize(),
                'amount': round(ingredient.get('quantity'), 2),
                'unitOfMeasurement': clean_unit(ingredient.get('measure')),
                'category': ingredient.get('foodCategory').capitalize()
            })
    description = get_description(name, ingredients)

    embeddings = get_embeddings(description)
    embeddings_json = json.dumps(embeddings)

    dish_dict = {
        'name': name,
        'image': image,
        'description': description,
        'ingredients': ingredients,
        'embeddings': embeddings_json,
    }

    insert_dish_and_ingredients(dish_dict)

    return dish_dict


def recognize_dish(image_path):
    import PIL.Image

    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    img = PIL.Image.open(image_path)
    model = genai.GenerativeModel('gemini-pro-vision')

    response = model.generate_content(
        ["In as less words as possible, no more than 8, name the dish in the image.", img],
        stream=True)
    response.resolve()
    print(response.text)
    return response.text



In [4]:
def db_insert_ingredient_if_not_exists(cursor, name, unit_of_measurement, category):
    cursor.execute("SELECT id FROM ingredients WHERE name = ?", (name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        cursor.execute("INSERT INTO ingredients (name, unitOfMeasurement, category) VALUES (?, ?, ?)",
                       (name, unit_of_measurement, category))
        return cursor.lastrowid


def db_insert_dish_if_not_exists(cursor, name, description, image, embeddings):
    cursor.execute("SELECT id FROM dishes WHERE name = ?", (name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        cursor.execute("INSERT INTO dishes (name, description, image, is_favorite, embeddings) VALUES (?, ?, ?, ?, ?)",
                       (name, description, image, False, embeddings))
        return cursor.lastrowid


def insert_dish_and_ingredients(dish):
    conn = sqlite3.connect('iis_db.sqlite')
    cursor = conn.cursor()
    try:
        # Insert the dish
        dish_id = db_insert_dish_if_not_exists(cursor, dish['name'], dish['description'], dish['image'],
                                            dish['embeddings'])

        # Insert ingredients and their relationship to the dish
        for ingredient in dish['ingredients']:
            ingredient_id = db_insert_ingredient_if_not_exists(cursor, ingredient['name'], ingredient['unitOfMeasurement'],
                                                            ingredient['category'])

            cursor.execute(
                "INSERT INTO dishes_ingredients (dishID, ingredientID, amount, unitOfMeasurement) VALUES (?, ?, ?, ?)",
                (dish_id, ingredient_id, ingredient['amount'], ingredient['unitOfMeasurement'],))

            conn.commit()
    except Exception as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    finally:
        cursor.close()
        conn.close()


def toggle_favorite(dish_name):
    conn = sqlite3.connect('iis_db.sqlite')
    cursor = conn.cursor()

    cursor.execute("SELECT id, is_favorite FROM dishes WHERE name = ?", (dish_name,))
    result = cursor.fetchone()
    if result:
        dish_id = result[0]
        is_favorite = not result[1]
        cursor.execute("UPDATE dishes SET is_favorite = ? WHERE id = ?", (is_favorite, dish_id))
        conn.commit()
    else:
        print("Dish not found.")

    cursor.close()
    conn.close()


def db_to_dict_dish(dish_name):
    # Connect to the SQLite database (or any other database you are using)
    conn = sqlite3.connect('iis_db.sqlite')
    cursor = conn.cursor()

    # Query to get the dish details
    cursor.execute('''
        SELECT name, image, description, is_favorite, embeddings 
        FROM dishes 
        WHERE name = ?''', (dish_name,))
    dish = cursor.fetchone()

    # If the dish does not exist, return None or an empty object
    if dish is None:
        print('Dish not found in the database')
        return None

    # Unpack the fetched dish data
    name, image, description, is_favorite, embeddings = dish

    # Query to get the ingredients for the dish
    cursor.execute('''
        SELECT i.name, di.amount, di.unitOfMeasurement, i.category
        FROM ingredients i
        JOIN dishes_ingredients di ON i.id = di.ingredientID
        JOIN dishes d ON di.dishID = d.id
        WHERE d.name = ?''', (dish_name,))

    # Fetch all ingredients
    ingredients = cursor.fetchall()

    # Format the ingredients
    formatted_ingredients = [
        {
            'name': name,
            'amount': amount,
            'unitOfMeasurement': unit_of_measurement,
            'category': category
        }
        for name, amount, unit_of_measurement, category in ingredients
    ]

    # Format the final dish object
    dish_details = {
        'name': name,
        'image': image,
        'description': description,
        'ingredients': formatted_ingredients,
        'embeddings': json.loads(embeddings),
        'is_favorite': is_favorite,
    }

    # Close the database connection
    conn.close()

    return dish_details


def get_favorites():
    conn = sqlite3.connect('iis_db.sqlite')
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM dishes WHERE is_favorite = 1")
    favorites_names = cursor.fetchall()

    cursor.close()
    conn.close()

    favorites = [db_to_dict_dish(favorite[0]) for favorite in favorites_names]
    for dish in favorites:
        del dish['embeddings']

    return favorites


def add_to_cart(items, add):
    conn = sqlite3.connect('iis_db.sqlite')
    cursor = conn.cursor()

    for item in items:
        ingredient_name = item.get('ingredient')
        amount_to_add_or_subtract = item.get('amount') if add else -item.get('amount')

        # Fetch the unit of measurement and category from the ingredients table
        cursor.execute("SELECT unitOfMeasurement, category FROM ingredients WHERE name = ?", (ingredient_name,))
        result = cursor.fetchone()
        if result:
            unit_of_measurement, category = result
        else:
            # Skip this item if it's not found in the ingredients table
            continue

        # Check if the ingredient is already in the cart
        cursor.execute("SELECT amount FROM cart WHERE name = ?", (ingredient_name,))
        result = cursor.fetchone()
        if result:
            # Ingredient is in the cart, calculate the new amount
            current_amount = result[0]
            new_amount = current_amount + amount_to_add_or_subtract

            if new_amount <= 0:
                # If the new amount is less than or equal to 0, remove the ingredient from the cart
                cursor.execute("DELETE FROM cart WHERE name = ?", (ingredient_name,))
            else:
                # Otherwise, update the ingredient's amount in the cart
                cursor.execute("UPDATE cart SET amount = ? WHERE name = ?", (new_amount, ingredient_name))
        elif amount_to_add_or_subtract > 0:
            # Ingredient is not in the cart and we're adding a positive amount, so insert it
            cursor.execute("INSERT INTO cart (name, amount, unitOfMeasurement, category) VALUES (?, ?, ?, ?)",
                           (ingredient_name, amount_to_add_or_subtract, unit_of_measurement, category))
        # Commit changes for each item to ensure data consistency
        conn.commit()

    cursor.close()
    conn.close()


def get_cart():
    conn = sqlite3.connect('iis_db.sqlite')
    cursor = conn.cursor()

    cursor.execute("SELECT name, amount, unitOfMeasurement, category FROM cart")
    cart_items = cursor.fetchall()

    cursor.close()
    conn.close()

    return [
        {
            'ingredient': name,
            'amount': amount,
            'unitOfMeasurement': unit_of_measurement,
            'category': category
        }
        for name, amount, unit_of_measurement, category in cart_items
    ]


def update_cart(items):
    conn = sqlite3.connect('iis_db.sqlite')
    cursor = conn.cursor()

    for item in items:
        ingredient_name = item.get('ingredient')
        new_amount = item.get('amount')

        if new_amount <= 0:
            cursor.execute("DELETE FROM cart WHERE name = ?", (ingredient_name,))
        else:
            cursor.execute("UPDATE cart SET amount = ? WHERE name = ?", (new_amount, ingredient_name))
        conn.commit()

    cursor.close()
    conn.close()


def empty_cart():
    conn = sqlite3.connect('iis_db.sqlite')
    cursor = conn.cursor()

    cursor.execute("DELETE FROM cart")
    conn.commit()

    cursor.close()
    conn.close()


def cosine_similarity(x, embeddings, n=4):
    similarities = []
    embeddings = [(name, json.loads(embedding)) for name, embedding in embeddings]

    # Calculate cosine similarity between x and each vector in embeddings
    for name, vector in embeddings:
        similarity = np.dot(x, vector) / (np.linalg.norm(x) * np.linalg.norm(vector))
        similarities.append((name, similarity))

    # Sort the similarities in descending order
    similarities.sort(key=lambda x: x[1], reverse=True)

    print(similarities[:4])

    # Return the names of the top two vectors with the highest cosine similarity
    return [sim[0] for sim in similarities[1:n + 1]]


def get_recommendations(dish_name):
    conn = sqlite3.connect('iis_db.sqlite')
    cursor = conn.cursor()

    cursor.execute("SELECT embeddings FROM dishes WHERE name = ?", (dish_name,))
    dish_embeddings = cursor.fetchone()
    if dish_embeddings:
        dish_embeddings = json.loads(dish_embeddings[0])
    else:
        print("Dish not found.")
        return []

    cursor.execute("SELECT name, embeddings FROM dishes")
    dishes = cursor.fetchall()

    cursor.close()
    conn.close()

    recommendations_names = cosine_similarity(dish_embeddings, dishes)
    recommendations = [db_to_dict_dish(recommendation) for recommendation in recommendations_names]

    return recommendations



In [10]:
# Inserting dishes

# for dish_name in dishes:
for dish_name in ['Couscous']:
    recipe = edamam_get_recipe(dish_name)
    print(recipe.get('hits')[0].get('recipe'))
    edamam_to_dish(recipe.get('hits')[0].get('recipe'))


{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_6e09585f7d574df5389377bc2e769b78', 'label': 'Couscous for One', 'image': 'https://edamam-product-images.s3.amazonaws.com/web-img/901/90106a5d68837b7fe9dec683a9ac9f0a?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEDsaCXVzLWVhc3QtMSJHMEUCIQDZeDPx4pcqLPPmen0intD9%2F%2BrioIiyfnY4P5VGv6aC5wIgQOf%2FuKS3sU3wUvmBfnJV9CaqH3neYgjXm9jbS0Y163squAUIYxAAGgwxODcwMTcxNTA5ODYiDNdtSM5e7q%2FwCpomdyqVBa3f0hVxjXJ%2Fkr7N3sCVhUjaobyKo%2BS9Cx0x7jUk5gU%2FLptbmvY8PqsR8NOHWmlLVNL%2BdHqzFay6T3zKg72OVKUiqY00qpdWhXd3Hhjjo%2F7b5MIWv4FZ3IEE7cHyZgmo8ihfrCiyMIVCfFMAehLUC8I4mw%2FSO%2Bv09p2%2FfhC4BD5u6%2BgCxK99A0mqrWgF6Tbu5meEnKdIsmhihO7PSlHjPguJHj33RDRLwni3O53pArCdaMz7TcDHm7ZK5hXTcgD5Q7HEuMcd%2FxFyM3Y9SEKEY7mGwvAFn6HmYw9oiZorMIG8Cpfu5rPk4z3rXZ5qngeXL2ghjgIVyuONt5trbXsORu7CU5rhE%2Buax9MPUJ%2FHoCu7%2FPQ2yRkzaNa55NPocqIcqtgdnTxvHLEvt9ekJh6qZPWEi9Nnk31P%2FgVBi78LLDUCKe%2F5KKlmQ2HATVWCdtrgonJzHxRwyKVE8JSjupFncOiom6Km3X5hXKqJeNTC59y6d2s1B1MfjtCTfqZYtdwoI2yRvgsvIcqt9EGj5xyM81I

In [6]:
dishes = [
    'Couscous',
    'Sauerbraten',
    'Sushi',
    'Tamales',
    'Baklava',
    'Jollof Rice',
    'Ceviche',
    'Bibimbap',
    'Spaghetti Carbonara',
    'Chicken Tikka Masala',
    'Kebab',
    'Tom Yum Goong',
    'Sashimi',
    'Risotto',
    'Falafel',
    'Pancakes',
    'Gumbo',
    'Ratatouille',
    'Arroz con Pollo',
    'Haggis',
    'Dim Sum',
    'Fish and Chips',
    'Poutine',
    'Mole',
    'Dolma',
    'Perogi',
    'Shakshuka',
    'Pirozhki',
    'Biryani',
    'Cordon Bleu',
    'Bratwurst',
    'Pupusas',
    'Tiramisu',
    'Fufu',
    'Pierogi',
    'Kabsa',
    'Gyro',
    'Arepas',
    'Gazpacho',
    'Lomo Saltado',
    'Kimchi Jjigae',
    'Samosa',
    'Fish Tacos',
    'Miso Soup',
    'Ravioli',
    'Peking Duck',
    'Lobster Thermidor',
    'Sauerbraten',
    'Tom Kha Gai',
    'Tapas',
    'Beef Wellington',
    'Cabbage Rolls',
    'Fesenjan',
    'Birria',
    'Mofongo',
    'Chicken Adobo',
    'Lángos',
    'Cacio e Pepe',
    'Lechón',
    'Tteokbokki',
    'Chicken Shawarma',
    'Feijoada',
    'Katsu Curry',
    'Gado-gado',
    'Jambalaya',
    'Meze',
    'Plov',
    'Chicken Paprikash',
    'Rösti',
    'Pastel de Nata',
    'Chiles en Nogada',
    'Bibingka',
    'Tostones',
    'Bánh Xèo',
    'Scotch Egg',
    'Kleftiko',
    'Hotpot',
    'Lutefisk',
    'Rasgulla',
    'Bobotie',
    'Calzone'
]


In [1]:
'yul' in 'yulit'

True

In [11]:
'yul'.capitalize()

'Yul'